<a href="https://colab.research.google.com/github/kirbyju/TCIA_Notebooks/blob/main/Accessing_and_visualizing_DICOM_datasets_hosted_on_TCIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary

Access to large, high quality data is essential for researchers to understand disease and precision medicine pathways, especially in cancer. However HIPAA constraints make sharing medical images outside an individual institution a complex process. [The Cancer Imaging Archive (TCIA)](https://www.cancerimagingarchive.net/) is a public service funded by the National Cancer Institute which addresses this challenge by providing hosting and de-identification services to take major burdens of data sharing off researchers. 

TCIA has published over 175 unique data collections containing more than 60 million images. Recognizing that images alone are not enough to conduct meaningful research, most collections are linked to rich supporting data including patient outcomes, treatment information, genomic / proteomic analyses, and expert image analyses (segmentations, annotations, and radiomic features). This notebook is focused on basic use cases for identifying TCIA datasets of interest and downloading them using command line tools and/or Python.

# 1 Learn about available Collections on the TCIA website

[Browsing Collections](https://www.cancerimagingarchive.net/collections) and [Analysis Results](https://www.cancerimagingarchive.net/tcia-analysis-results/) datasets on TCIA are the easiest ways to become familiar with what is available.  These pages will help you quickly identify datasets of interest, find valuable supporting data that are not available via our APIs (e.g. clinical spreadsheets, non-DICOM segmentation data), and answer most common questions you might have about the datasets.  

# 2 Downloading with NBIA Data Retriever

TCIA utilizes software called NBIA to manage its DICOM data.  One way to download TCIA data is to install the [linux command-line version of the NBIA Data Retriever](https://wiki.cancerimagingarchive.net/x/2QKPBQ) using the following steps.  This tool provides a number of useful features such as multi-threaded downloads, auto-retry if there are any problems, saving data in an organized hierarchy on your hard drive (Collection > Patient > Study > Series > Images) and providing a CSV file containing key DICOM metadata about the images you've downloaded.

### 2.1 Install the NBIA Data Retriever CLI package

In [ ]:
# install NBIA Data Retriever CLI software for downloading images later in this notebook

!mkdir /usr/share/desktop-directories/
!wget -P /content/NBIA-Data-Retriever https://cbiit-download.nci.nih.gov/nbia/releases/ForTCIA/NBIADataRetriever_4.4/nbia-data-retriever-4.4.deb
!dpkg -i /content/NBIA-Data-Retriever/nbia-data-retriever-4.4.deb

# NOTE: If you're working on a Linux OS that uses RPM packages you can try changing the wget line above to point to
#       https://cbiit-download.nci.nih.gov/nbia/releases/ForTCIA/NBIADataRetriever_4.4/NBIADataRetriever-4.4-1.x86_64.rpm

### 2.2 Download a manifest file
The Data Retriever software works by ingesting a "manifest" file that contains the DICOM Series Instance UIDs of the scans you'd like to download. Let's assume that after [Browsing the Collections](https://www.cancerimagingarchive.net/collections) you decided you were interested in the [RIDER Breast MRI](https://doi.org/10.7937/K9/TCIA.2015.H1SXNUXL) Collection.  If you're working from your local machine you can simply click the blue "Download" button on the [RIDER Breast MRI](https://doi.org/10.7937/K9/TCIA.2015.H1SXNUXL) page to save the manifest file to your computer.  If you're working on Google Colab or some other remote server the easiest thing to do is use wget to save it to your VM as shown below.



In [ ]:
# use wget to download the manifest

!wget -O /content/RIDER-Breast-MRI.tcia https://wiki.cancerimagingarchive.net/download/attachments/22512757/doiJNLP-Fo0H1NtD.tcia?version=1&modificationDate=1534787017928&api=v2


In [ ]:
# OPTIONAL: If you're just looking for a quick demo, you can run this command 
# to edit the manifest file to download only the first 3 scans

!head -n 9 /content/RIDER-Breast-MRI.tcia > /content/RIDER-Breast-MRI-Sample.tcia

### 2.3 Open the manifest file with NBIA Data Retriever
Next let's open the sample manifest file with Data Retriever to download the actual DICOM data.

***Note: After running the following command you have to click in the output cell, type "y" and press Enter to agree with the TCIA Data Usage Policy to start the download.***

In [ ]:
# download the data using NBIA Data Retriever

!/opt/nbia-data-retriever/nbia-data-retriever --cli '/content/RIDER-Breast-MRI-Sample.tcia' -d /content/ 


### 2.4 Review the downloaded data
You should now find that the data have been saved to your machine in a well organized hierarchy with some useful metadata in the accompanying CSV file and a license file detailing how it can be used.  Take a second to go check it out before moving on!

### 2.5 Downloading "Limited-Access" Collections with the Data Retriever
In some cases you must specifically request access to [Collections](https://www.cancerimagingarchive.net/collections/) before you can download them.  Information about how to do this can be found on the homepage for the Collection(s) that you're interested in, but will always require that you first [create a TCIA user account](https://wiki.cancerimagingarchive.net/x/xgHDAg.).  Once you've created an account you can use your login/password to create the credential file that NBIA Data Retriever uses to verify your permissions.

In [ ]:
# Create the credential file
# NOTE: You must enter your real user name and password before you run this,
# or edit the resulting text file with your real credentials after it's created.

lines = ['userName=YourUserName', 'passWord=YourPassword']
with open('credentials.txt', 'w') as f:
    f.write('\n'.join(lines))

Let's say that we're interested in the [RIDER Neuro MRI](http://doi.org/10.7937/K9/TCIA.2015.VOSN3HN1) Collection. As you can see on the Collection page, you must sign and submit a TCIA Restricted License Agreement to help@cancerimagingarchive.net before accessing the data. Once you've done this, click the blue Download button on the RIDER Neuro MRI page to save the manifest file to your computer or grab it by using the wget command shown below.

In [ ]:
# download a manifest with the restricted data
!wget -O /content/RIDER_Neuro_MRI.tcia https://wiki.cancerimagingarchive.net/download/attachments/22512753/TCIA_RIDER_NEURO_MRI_06-22-2015.tcia?version=1&modificationDate=1534787443910&api=v2


In [ ]:
# OPTIONAL: If you're just looking for a quick demo, you can run this command 
# to edit the manifest file to download only the first 3 scans

!head -n 9 /content/RIDER_Neuro_MRI.tcia > /content/RIDER_Neuro_MRI-Sample.tcia

Now let's open the manifest file with NBIA Data Retriever to download your data.This time we're also invoking the "-l" parameter to tell it where you saved your credential file.

***Note: After running the following command you have to click in the output cell, type "y" and press Enter to agree with the TCIA Data Usage Policy to start the download.***

In [ ]:
# download the data using NBIA Data Retriever
# you may need to update the path to your credential file

!/opt/nbia-data-retriever/nbia-data-retriever --cli '/content/RIDER_Neuro_MRI-Sample.tcia' -d /content/ -l /content/credentials.txt

# 3 Searching and downloading data via the REST APIs 
NBIA REST APIs are provided to the search and download functions used in the TCIA radiology portal, and allow access to both public and limited access collections.
1. The [NBIA Search REST APIs](https://wiki.cancerimagingarchive.net/x/fILTB) allow you to perform basic queries and download data from **public** collections. This API does not require a TCIA account.
2. The [NBIA Search with Authentication REST APIs](https://wiki.cancerimagingarchive.net/x/X4ATBg) allow you to perform basic queries and download data from **public and limited-access** collections. This API requires a TCIA account for creation of authentication tokens.
3. The [NBIA Advanced REST APIs](https://wiki.cancerimagingarchive.net/x/YoATBg) also allow access to **public and limited-access** collections, but provides query endpoints mostly geared towards developers seeking to integrate searching and downloading TCIA data into web and desktop applications.  This API requires a TCIA account for creation of authentication tokens.

Thise notebook will focus on the fully public [NBIA Search REST APIs](https://wiki.cancerimagingarchive.net/x/fILTB).  If you'd like to see examples using the other APIs check out [this notebook](https://github.com/kirbyju/TCIA_Notebooks/blob/main/ACNS0332/ACNS0332.ipynb) which shows many similar examples with the additional steps necessary to create a secure token using your TCIA login credentials.

***Note:*** Many of the examples below allow for additional query parameters to refine your search results.  These are covered in the documentation links above.

### 3.1 Explore public data with REST API Queries
In this section we'll look at how to query and download public data via the [NBIA Search REST APIs](https://wiki.cancerimagingarchive.net/x/fILTB).  The URL for accessing the Search APIs changes slightly depending on whether or not you would like to access the [National Lung Screening Trial (NLST)](https://doi.org/10.7937/TCIA.HMQ8-J677) collection, which lives on its own server due to its size (26,000+ patients, ~13 TBytes).  Here are the base URLs:

* All other Collections - https://services.cancerimagingarchive.net/nbia-api/services/v1/
* NLST - https://services.cancerimagingarchive.net/nlst-api/services/v1/




In [ ]:
# set API base URLs

base_url = "https://services.cancerimagingarchive.net/nbia-api/services/v1/"
nlst_url = "https://services.cancerimagingarchive.net/nlst-api/services/v1/"

# imports

import requests
import pandas as pd
import json

#### 3.1.1 List collections and determine modalities and body parts
Let's run a query to see what Collections are available.  After obtaining a list we'll use some example queries to learn more about specific Collections.

In [ ]:
# get list of available collections as JSON

data_url = base_url + "getCollectionValues"
data = requests.get(data_url).json()
print(json.dumps(data, indent=2))

Let's choose a Collection from the list above and find out more about what modalities and body parts it contains.  We'll define these as functions so we can use them as part of more complex queries later in the notebook.

In [ ]:
# Define a function to return modalities for a collection as JSON

# Choose a collection of interest
collection = "TCGA-LUAD"

def getModality(collection):
    data_url = base_url + "getModalityValues?Collection=" + collection
    data = requests.get(data_url)
    if data.text != "":
        return data.json()
    else:
        print("Collection not found.")

getModality(collection)

In [ ]:
# Define a function to return body parts examined for a collection as JSON
collection = "TCGA-LUAD"

def getBodyPart(collection):
    data_url = base_url + "getBodyPartValues?Collection=" + collection
    data = requests.get(data_url)
    if data.text != "":
        return data.json()
    else:
        print("Collection not found.")

getBodyPart(collection)

#### 3.1.2 Exploring patient, study and scan metadata
You can use the /getPatient endpoint to obtain details about species, gender, and ethnicity where available for a given collection. You can also learn whether a [phantom](https://www.nist.gov/physics/what-are-imaging-phantoms) subject or not.

In [ ]:
# Retrieve patient details as JSON and create pandas dataframe w/ optional file export

# Choose a collection of interest
collection="CPTAC-CCRCC"

data_url = base_url + "getPatient?Collection=" + collection
data = requests.get(data_url)

if data.text != "":
    df = pd.DataFrame(data.json())
    display(df)
    # optional - save to JSON or CSV file
    df.to_csv(collection+'_patient_metadata.csv')
    # df.to_json(collection+'_patient_metadata.json')
else:
    print("Collection not found.")


Here's an example that does the same thing with the NLST collection, which is living on its own server and uses the slightly modified API URL.  Any of the other queries shown in the notebook should work simply by setting the collection variable to "NLST" and updating "base_url" to the "nlst_url" in the "data_url" variable as shown here.

In [ ]:
# Retrieve patient details as JSON and create pandas dataframe w/ optional file export

# Choose a collection of interest
collection="NLST"

# NOTE: we are using the nlst_url variable rather than the general base_url
data_url = nlst_url + "getPatient?Collection=" + collection
data = requests.get(data_url)

if data.text != "":
    df = pd.DataFrame(data.json())
    display(df)
    # optional - save to JSON or CSV file
    df.to_csv(collection+'_patient_metadata.csv')
    # df.to_json(collection+'_patient_metadata.json')
else:
    print("Collection not found.")

The /getStudy endpoint can be used to obtain study/visit details such as the anonymized study date, subject's age at the time of visit, number of scans acquired each timepoint and more.

In [ ]:
# getStudy details for collection and create pandas dataframe w/ optional file export

collection="CPTAC-CCRCC"

data_url = base_url + "getPatientStudy?Collection=" + collection
data = requests.get(data_url)

if data.text != "":
    df = pd.DataFrame(data.json())
    display(df)
    # optional - save to JSON or CSV file
    df.to_csv(collection+'_study_metadata.csv')
    # df.to_json(collection+'_study_metadata.json')
else:
    print("Collection not found.")

We can also create reports that give useful metadata about each scan in the dataset (e.g. series description, modality, scanner manufacturer & software version, number of images).  We'll define a function for this one as well so we can use the JSON output in a more complex query later.  You must choose a collection, but modality is optional.

In [ ]:
# Define a function to return scan/series metadata for a collection as JSON 
# modality is optional

collection = "LIDC-IDRI"
modality = ""

def getSeries(collection, modality=""):
    if modality != "":
        data_url = base_url + "getSeries?Collection=" + collection + "&Modality=" + modality
        data = requests.get(data_url)
        if data.text != "":
            return data.json()
        else:
            print("No results: Please check to make sure the Collection " + collection + " exists and it contains " + modality + " modality.")
    else:
        data_url = base_url + "getSeries?Collection=" + collection
        data = requests.get(data_url)
        if data.text != "":
            return data.json()
        else:
            print("Collection not found.")

getSeries(collection, modality)

Let's save the output from that function in a dataframe so it's easier to view, analyze and export to CSV (if desired).

In [ ]:
collection = "LIDC-IDRI"
modality = ""

# call previously created getSeries function for a given Collection
data = getSeries(collection, modality)

# load it to a dataframe
df = pd.DataFrame(data)
display(df)

# optional - save to JSON or CSV file
#df.to_csv(collection+'_scan_metadata.csv')
# df.to_json(collection+'_scan_metadata.json')

#### 3.1.3 Advanced queries
Here are some additional examples which can be useful to address common questions about TCIA's datasets.  These are where we will rely on some of the functions we defined earlier in the notebook.

In [ ]:
# Find out how many patients, which modalities and body parts are in a collection

# set collection of interest
collection = "QIN-PROSTATE-Repeatability"

# get list of patients in Collection
data_url = base_url + "getPatient?Collection=" + collection
data = requests.get(data_url)

if data.text != "":
    # get modalities for collection
    modalities = getModality(collection)
    clean_modalities = set(item['Modality'] for item in modalities)
    # get body parts for collection
    bodyParts = getBodyPart(collection)
    clean_bodyParts = set()
    # replace null bodyParts with "Not Specified"
    for item in bodyParts:
        if len(item):
            clean_bodyParts.add(item['BodyPartExamined'])
        else:
            clean_bodyParts.add('Not Specified')
    # print cleaned up results
    print(collection, 'has', len(data.json()), 'patients,',
        clean_modalities, 'modalities, and',
        clean_bodyParts, 'anatomic entities')
else:
    print("Collection not found.")

In [ ]:
# Calculate summary statistics for a given collection 
collection = "CPTAC-LSCC"

# Call the getSeries function we created above
data = getSeries(collection)

# convert the output to dataframe
df = pd.DataFrame(data)

# Summarize patients
print('Summary Statistics\n')
print('Subjects: ', len(df['PatientID'].value_counts()), 'subjects')
print('Subjects: ', len(df['StudyInstanceUID'].value_counts()), 'studies')
print('Subjects: ', len(df['SeriesInstanceUID'].value_counts()), 'series')
print('Images: ', df['ImageCount'].sum(), 'images\n')

# Summarize modalities
print("Series Counts - Modalities:")
print(df['Modality'].value_counts(dropna=False),'\n')

# Summarize body parts
print("Series Counts - Body Parts Examined:")
print(df['BodyPartExamined'].value_counts(dropna=False),'\n')

# Summarize manufacturers
print("Series Counts - Device Manufacturers:")
print(df['Manufacturer'].value_counts(dropna=False))

In [ ]:
# get patient counts for a given modality across all Collections
# this is particularly useful for finding Collections w/ segmentation labels (SEG/RTSTRUCT)

modality = "SEG"

data_url = base_url + "getCollectionValues"
data = requests.get(data_url)

if data.text != "":
    notFound=[]
    data = data.json()
    for x in data:
        collection = x['Collection']
        patient_url = base_url + "getPatientByCollectionAndModality?Collection=" + collection + "&Modality=" + modality
        patients = requests.get(patient_url)
        if patients.text != "":
            patients = patients.json()
            print(collection, 'has', len(patients), 'patients with', modality, 'modality') 
        else:
            notFound.append(collection)
    print('The following collections have no patients with', modality, 'modality:', notFound)
else:
    print("Modality not found.")

In [ ]:
# create a dataframe that shows patient counts, modalities, body parts for all collections
# note: this can take >6 minutes to run

resultsList = []

# get list of available collections
collection_url = base_url + "getCollectionValues"
collection_data = requests.get(collection_url).json()

# loop through list of collections to populate dataframe
for x in collection_data:
    collectionName = x['Collection']
    patient_url = base_url + "getPatient?Collection=" + collectionName
    patients = requests.get(patient_url).json()
    clean_PatientIds = set(item['PatientId'] for item in patients)
    patientCount = len(clean_PatientIds)
    modality_url = base_url + "getModalityValues?Collection=" + collectionName
    modalities = requests.get(modality_url).json()
    clean_modalities = set(item['Modality'] for item in modalities)
    bodyPart_url = base_url + "getBodyPartValues?Collection=" + collectionName
    bodyParts = requests.get(bodyPart_url).json()
    clean_bodyParts = set()
    for item in bodyParts:
        if len(item):
            clean_bodyParts.add(item['BodyPartExamined'])
        else:
            clean_bodyParts.add('Not Specified')
    data = [collectionName, patientCount, clean_modalities, clean_bodyParts]
    resultsList.append(data)
    df = pd.DataFrame(columns=['Collection', 'Subjects', 'Modalities', 'BodyParts'], data=resultsList)
    
display(df)

# optional export to CSV
# df.to_csv('collection_metadata.csv')

### 3.2 Downloading data with the REST API
Here are some examples of how to download data from ***public collections*** using the API.  In examples that would create large downloads there is some code included to limit the download to the first few series.  This can be commented out if you're ready to download full datasets.

Remember, if you'd like to see examples for downloading a ***restricted collection*** check out [this notebook](https://github.com/kirbyju/TCIA_Notebooks/blob/main/ACNS0332/ACNS0332.ipynb) which shows examples with the additional steps necessary to create a secure token using your TCIA login credentials.

In [ ]:
import pandas as pd
import requests, zipfile
from io import BytesIO

First let's look at how to download an individual series as a zip file based on a SeriesInstanceUID and print some useful information about it with the "getSeriesMetaData" endpoint. In particular, the Data Description URI included in the Series Metadata output takes you to the home page for the dataset where you can find valuable supporting data that are not available via our APIs (e.g. clinical spreadsheets, non-DICOM segmentation data) and find answers to common questions you might have about the dataset.

In [ ]:
# example PET scan
seriesInstanceUID="1.3.6.1.4.1.14519.5.2.1.7009.2401.163884424533721336163307900458"
data_url = base_url + "getImage?SeriesInstanceUID=" + seriesInstanceUID
request = requests.get(data_url)
file = zipfile.ZipFile(BytesIO(request.content))
print("File list:",file.namelist())
file.extractall(path = "apiDownload/" + seriesInstanceUID)

metadata_url = base_url + "getSeriesMetaData?SeriesInstanceUID=" + seriesInstanceUID
metadata = requests.get(metadata_url).json()
print("Series Metadata:",metadata)


Now let's define a generic download function that we can re-use for the remaining use cases.  This will take a list of series UIDs as the input, download each scan, and create a dataframe/CSV that contains the metadata about each of those scans.  It also accepts an optional parameter to specify a file name if you'd like a CSV export of the dataframe.

In [ ]:
# define a function to accept a list of seriesInstanceUIDs and download it
# reminder: this only downloads the first 3 scans unless you comment out that section

def downloadSeries(series_data, csv_filename=""):  
    manifestDF=pd.DataFrame()
    seriesUID = ''
    count = 0
    for x in series_data:
        seriesUID = x['SeriesInstanceUID']
        data_url = base_url + "getImage?SeriesInstanceUID=" + seriesUID
        print("Downloading " + data_url)
        data = requests.get(data_url)
        file = zipfile.ZipFile(BytesIO(data.content))
        # print(file.namelist())
        file.extractall(path = "apiDownload/" + collection + "/" + seriesUID)
        # write the series metadata to a dataframe
        metadata_url = base_url + "getSeriesMetaData?SeriesInstanceUID=" + seriesUID
        metadata = requests.get(metadata_url).json()
        newRow = pd.DataFrame.from_dict(metadata)
        tmpManifest = pd.concat([manifestDF, newRow], ignore_index = True)
        tmpManifest.reset_index()
        manifestDF = tmpManifest
        # Repeat n times for demo purposes - comment out these next 3 lines to download a full results
        count += 1;
        if count == 3:
            break  
    # display manifest dataframe and/or save manifest to CSV file
    if csv_filename != "":
        manifestDF.to_csv(csv_filename + '.csv')
        display(manifestDF)
    else:
        display(manifestDF)

Let's use the getSeries function we created earlier together with the downloadSeries function to download an entire collection (or just the first 3 scans if you didn't modify the downloadSeries function). 

In [ ]:
# choose collection
collection = "NSCLC-Radiomics"

# call getSeries function to retrieve scan metadata for the whole collection
series_data = getSeries(collection)

# feed series_data to our downloadSeries function
downloadSeries(series_data, collection + "_full_Collection")

We can also leverage the flexability to specify Collection + modality for the getSeries function and feed that subset to the downloadSeries function.

In [ ]:
# choose collection
collection = "NSCLC-Radiomics"
modality = "SEG"

# call getSeries function to retrieve scan metadata for the whole collection
series_data = getSeries(collection, modality)

# feed series_data to our downloadSeries function
downloadSeries(series_data, collection + "_full_Collection")

It's possible to use https://nbia.cancerimagingarchive.net to create a "[Shared Cart](https://wiki.cancerimagingarchive.net/display/NBIA/TCIA+Radiology+Portal+User+Guide#TCIARadiologyPortalUserGuide-SharingDatainYourCart)" which includes a specific set of scans you'd like to share with others. After creating a Shared Cart you receive a URL like https://nbia.cancerimagingarchive.net/nbia-search/?saved-cart=nbia-49121659384603347 which can be shared with others.  Try clicking the link to see what this looks like on the TCIA website.  Then use the code below to see how you can use the cart name to download the (first 3) related scans via our API.

In [ ]:
# Download a "Shared Cart" that has been previously created via the NBIA GUI (https://nbia.cancerimagingarchive.net)

cartName="nbia-49121659384603347"

cart_URL = base_url + "getContentsByName?name=" + cartName
series_data = requests.get(cart_URL).json()

# feed series_data to our downloadSeries function
downloadSeries(series_data, collection + "_full_Collection")

### 3.3 Organizing the data (optional)
The previous code saves all series in a directory named with the Series Instance UID.  This may be fine for some use cases, but often it is helpful to organize the data in a more descriptive fashion.  Below we'll take advantage of the open source [dicomsort](https://github.com/pieper/dicomsort) tool to save things in a Patient ID > Study Date > Series UID > Image hierachy with descriptive names.

In [ ]:
# download https://github.com/pieper/dicomsort and install dependencies

!pip install pydicom
!wget https://raw.githubusercontent.com/pieper/dicomsort/master/dicomsort.py

In [ ]:
# use dicomsort to organize the data
# note: the -s flag creates a symlink instead of making a second copy the data

!python dicomsort.py -s apiDownload organizedDICOM/%PatientID/%StudyDate/%SeriesDescription-%SeriesInstanceUID/%InstanceNumber.dcm

# 4 Data Visualization




Server-side 2D visualization of specific scans in a browser is possible by using our OHIF Viewer if you know the Series Instance UID of the scan you'd like to view.  For example: https://nbia.cancerimagingarchive.net/viewer/?series=1.3.6.1.4.1.14519.5.2.1.7311.5101.225402322918877902268269283832.  

NCI also funds a project called the [Imaging Data Commons](https://portal.imaging.datacommons.cancer.gov/), which can be useful for visualizing and analyzing TCIA's public datasets.  They currently do not host any of our limited-access Collections.  

You can use the code below to pull up specific studies/series of interest using a StudyInstanceUID or SeriesInstanceUID.  The significant advantage to using the IDC viewer is that their version of OHIF supports loading entire studies (not just individual series) and also overlaying annotations/segmentations.  After entering a UID of interest, run the cell below and click the link to open up their viewer in your browser.

In [ ]:
# helper function to view a study or a specific series hosted by IDC
def get_idc_viewer_url(studyUID, seriesUID=None):
  url = "https://viewer.imaging.datacommons.cancer.gov/viewer/"+studyUID
  if seriesUID is not None:
    url = url+"?seriesInstanceUID="+seriesUID
  return url

print(get_idc_viewer_url("1.3.6.1.4.1.14519.5.2.1.6279.6001.321085339464682432111441689315"))

It may also be desirable to look at the data directly within your  notebook.  If you're interested in that, please check out **(link coming soon)** to learn about how to visualize TCIA data using [itkwidgets](https://github.com/InsightSoftwareConsortium/itkwidgets).

# Acknowledgements
TCIA is funded by the [Cancer Imaging Program (CIP)](https://imaging.cancer.gov/), a part of the United States [National Cancer Institute (NCI)](https://www.cancer.gov/), and is managed by the [Frederick National Laboratory for Cancer Research (FNLCR)](https://frederick.cancer.gov/).

This notebook was created by [Justin Kirby](https://www.linkedin.com/in/justinkirby82/) and Qinyan Pan.  If you leverage this notebook or any TCIA datasets in your work please be sure to comply with the [TCIA Data Usage Policy](https://wiki.cancerimagingarchive.net/x/c4hF). In particular, make sure to cite the DOI(s) for the specific TCIA datasets you used in addition to the following paper!

## TCIA Citation

Clark, K., Vendt, B., Smith, K., Freymann, J., Kirby, J., Koppel, P., Moore, S., Phillips, S., Maffitt, D., Pringle, M., Tarbox, L., & Prior, F. (2013). The Cancer Imaging Archive (TCIA): Maintaining and Operating a Public Information Repository. Journal of Digital Imaging, 26(6), 1045–1057. https://doi.org/10.1007/s10278-013-9622-7